In [1]:
import gc
import pickle
import time

##--get graph map
def get_graph_info(gf_file):
    '''
    get graph information as a map
    key: user id, value: list of friend id
    '''
    gf_map = {}
    with open(gf_file, 'r') as cf:
        for line in cf:
            if line[0] != '#':
                node_list = line.split('\t')
                if len(node_list) == 2:
                    node_list = [int(id) for id in node_list]
                    if node_list[0] in gf_map:
                        gf_map[node_list[0]].add(node_list[1])
                    else:
                        gf_map[node_list[0]] = set([node_list[1]])
                    if node_list[1] in gf_map:
                        gf_map[node_list[1]].add(node_list[0])
                    else:
                        gf_map[node_list[1]] = set([node_list[0]])                    
                else:
                    print "wrong format line: ", line
    return gf_map

def get_comm_info(comm_file):
    '''
    get community information, two maps
    map1: key: user id, value: community id array
    map2: key: community id, value: user id array
    '''
    comm_map_usr = {}
    comm_map_comm = {}
    comm_id = 0
    with open(comm_file, 'r') as cf:
        for line in cf:
            node_list = line.split('\t')
            node_list = [int(id) for id in node_list]
            for id in node_list:
                if id in comm_map_usr:
                    comm_map_usr[id].append(comm_id)
                else:
                    comm_map_usr[id] = [comm_id]
            comm_map_comm[comm_id] = node_list[:]
            comm_id += 1
    return comm_map_usr, comm_map_comm


In [ ]:
#t0=time.time()
#gf_file = "../data/com-lj.ungraph.txt"
#gf_map = get_graph_info(gf_file)
#gc.collect()
#print "Get graph map"
#gf_file = "whole_graph_map.pkl"
#with open(gf_file, 'wb') as fl:
#    pickle.dump(gf_map, fl)
#print "Dump the whole graph map to pickle"
#gc.collect()


In [2]:
import time
t0=time.time()
gf_file = "whole_graph_map.pkl"
file = open(gf_file,'rb')
gf_map = pickle.load(file)
file.close()

print time.time()-t0

181.843771935


In [3]:
t0=time.time()

##--get community information
comm_file = '../data/com-lj.all.cmty.txt'
comm_map_usr, comm_map_comm = get_comm_info(comm_file)

usr_to_com, com_to_usr = comm_map_usr, comm_map_comm 



In [4]:
del gf_map

In [6]:
import snap
import numpy as np
import pandas as pd
import random
import copy
import time
random.seed(2016)
import scipy.stats
t0=time.time()


G= snap.LoadEdgeList(snap.PUNGraph, "../data/com-lj.ungraph.txt", 0, 1)
print "Time Elapsed: ", time.time()-t0, "second"
print "Number of Nodes: ", G.GetNodes()
print "Number of Edges: ", G.GetEdges()



Time Elapsed:  21.5357861519 second
Number of Nodes:  3997962
Number of Edges:  34681189


In [7]:
nodes = [node.GetId() for node in G.Nodes()]
print len(nodes)

3997962


In [8]:
edges = [edge.GetId() for edge in G.Edges()]
print len(edges)

34681189


In [9]:
extended_com_to_usr = {com : com_to_usr[com] for com in com_to_usr}  
extended_usr_to_com = {usr : usr_to_com[usr] for usr in usr_to_com}  
usr_belong_to_com = set(usr_to_com.keys())
n = len(com_to_usr)

for usr in nodes:
    if usr in usr_belong_to_com:
        continue
    # if a user does not belong to any community,
    # create a community consisting of this user
    extended_com_to_usr[n] = [usr]  
    extended_usr_to_com[usr] = [n]
    n+=1

In [10]:
print "number of communities: ", len(comm_map_usr)
print "number of users belongig to at least one community: ", len(comm_map_comm)

print "number of extended communities: ", len(extended_com_to_usr)
print "number of extended community users: ", len(extended_usr_to_com)

number of communities:  1147948
number of users belongig to at least one community:  664414
number of extended communities:  3514428
number of extended community users:  3997962


# Use dictionary to store the number of edges between two communities (gave up)

In [67]:

t0 = time.time()

nedges_between_comms={}
count = 0 
for edge in edges:
    if count >10**4:
        break
    (usr1,usr2) = edge
    for comm1 in extended_usr_to_com[usr1]:
        for comm2 in extended_usr_to_com[usr2]:
            comm1, comm2 = min(comm1,comm2), max(comm1,comm2)
            if (comm1,comm2) in nedges_between_comms:
                nedges_between_comms[comm1,comm2]+=1
            else:
                nedges_between_comms[comm1,comm2]=1
    
    count+=1

print "{0:02f}".format(time.time()-t0)

9.732694


# Use sparse matrix to store the number of edges between two communities

## Results: 10^7 edges done after 13 hours, memory used reach 27GB. 
## All of the 3*10^7 edges done after 20.3 hours, memory used reach 30.5GB

In [51]:
import numpy as np
#from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix


In [66]:
ncomms = len(extended_com_to_usr)
#A = csr_matrix((ncomms, ncomms), dtype=np.int8)
A = lil_matrix((ncomms, ncomms), dtype=np.int8)

t0 = time.time()


count = 0 
k=1
for edge in edges:
    if count >10**8:
        break
    (usr1,usr2) = edge
    for comm1 in extended_usr_to_com[usr1]:
        for comm2 in extended_usr_to_com[usr2]:
            comm1, comm2 = min(comm1,comm2), max(comm1,comm2)
            A[comm1,comm2]+=1

    
    count+=1
    if count==10**k:
        print count, "Edges done, Time Elapsed:", "{0:02f}".format(time.time()-t0)
        k+=1

print "{0:02f}".format(time.time()-t0)

10 Edges done, Time Elapsed: 0.000598
100 Edges done, Time Elapsed: 0.104770
1000 Edges done, Time Elapsed: 0.366600
10000 Edges done, Time Elapsed: 142.600652
100000 Edges done, Time Elapsed: 997.903907
1000000 Edges done, Time Elapsed: 4942.929437
10000000 Edges done, Time Elapsed: 46031.591431
73284.256369


In [69]:
A

<3514428x3514428 sparse matrix of type '<type 'numpy.int8'>'
	with 352749634 stored elements in LInked List format>

In [ ]:
import cPickle as pickle
import numpy as np
import scipy.sparse


with open('sparse_array.dat', 'wb') as outfile:
    pickle.dump(A, outfile, pickle.HIGHEST_PROTOCOL)